In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.applications import MobileNet

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
img_width = 150
img_height = 150

In [3]:
train_datagen=ImageDataGenerator(rescale = 1./255, horizontal_flip=True) #included in our dependencies

train_generator=train_datagen.flow_from_directory('Train', # this is where you specify the path to the main data folder
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

validation_generator=train_datagen.flow_from_directory('Test', # this is where you specify the path to the main data folder
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 9600 images belonging to 48 classes.
Found 2608 images belonging to 48 classes.


In [ ]:
base_model=MobileNet(input_shape=(150, 150, 3), include_top = False, weights = 'imagenet')

33808384/58889256 [================>.............] - ETA: 2:00

In [5]:
feature_batch = base_model(train_generator[0])
print(feature_batch.shape)

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\User\AppData\Roaming\Python\Python36\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(32, 5, 5, 1280)


In [7]:
base_model.trainable = False

In [8]:
flatten = Flatten()

(32, 1280)


In [9]:
prediction_layer = Dense(48,activation ='softmax')

In [10]:
model = Sequential([
  base_model,
  flatten,
  prediction_layer
])


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 48)                61488     
Total params: 2,319,472
Trainable params: 61,488
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

# scheduler = LearningRateScheduler(schedule, verbose=0) # schedule is a function

logdir = os.path.join('callbacks')
tensorboard = TensorBoard(log_dir = logdir, profile_batch = 100000000, histogram_freq = 0, write_graph = True, write_images = True)




stop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

checkpoint = ModelCheckpoint('./model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//validation_generator.batch_size
model.fit_generator(train_generator, 
    steps_per_epoch = step_size_train, 
    epochs = 1, validation_data = validation_generator, 
    validation_steps = step_size_validation,
         callbacks=[stop, checkpoint, tensorboard])

C:\Users\User\AppData\Roaming\Python\Python36\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\User\AppData\Roaming\Python\Python36\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 49/300 [===>..........................] - ETA: 36:53 - loss: 4.1919 - accuracy: 0.0249